In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps


In [2]:
# Import csv and save as dataframe
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Chippewa Falls,US,44.94,-91.39,82.99,65,98,1.01,light rain,0.0,0
1,1,Zaysan,KZ,47.47,84.87,63.95,51,2,10.09,clear sky,0.0,0
2,2,Parakai,NZ,-36.65,174.43,48.00,93,8,1.01,light rain,0.0,0
3,3,Avarua,CK,-21.21,-159.78,71.60,68,98,13.87,overcast clouds,0.0,0
4,4,Punta Arenas,CL,-53.15,-70.92,30.20,86,40,6.93,scattered clouds,0.0,0


In [3]:
# Filter the DataFrame for minimum and maximum temperature preferences, and if the rain or snow accumulation is 0 inches or not using conditional statements. Do the following:

# Prompt the customer for the minimum temperature preference.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the customer for the maximum temperature preference.
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

# Prompt the customer to answer if he or she would like it to be raining or not, using input("Do you want it to be raining? (yes/no) ").
raining = str(input("Do you want it to be raining? (yes/no) "))


# Prompt the customer to answer if he or she would like it to be snowing or not, using input("Do you want it to be snowing? (yes/no) ").
snowing = str(input("Do you want it to be snowing? (yes/no) "))


# Filter the dataset to find the cities that fit the criteria.
# THE CODE BELOW ASSUMES THAT THE USER WILL ENTER yes/no IN CORRECTLY.  
# IT WILL NOT VERIFIY THE DATA HAS BEEN ENTERED CORRECTLY

if raining == "yes":
    if snowing == "yes":
        preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                        (city_data_df["Rain (inches)"] > 0) & \
                                        (city_data_df["Snow (inches)"] > 0)]
    elif snowing == "no":
        preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                        (city_data_df["Rain (inches)"] > 0) & \
                                        (city_data_df["Snow (inches)"] == 0)]
elif snowing == "yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                        (city_data_df["Rain (inches)"] > 0) & \
                                        (city_data_df["Snow (inches)"] > 0)]
elif snowing == "no":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                        (city_data_df["Rain (inches)"] == 0) & \
                                        (city_data_df["Snow (inches)"] == 0)]

preferred_cities_df.head(10)

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 75
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
3,3,Avarua,CK,-21.21,-159.78,71.60,68,98,13.87,overcast clouds,0.0,0
6,6,Mandalgovi,MN,45.76,106.27,70.16,44,99,14.81,overcast clouds,0.0,0
15,15,Pringsewu,ID,-5.36,104.97,73.65,94,89,2.53,overcast clouds,0.0,0
44,44,Liwale,TZ,-9.77,37.93,70.50,82,84,3.58,light rain,0.0,0
60,60,Poum,NC,-20.23,164.02,70.12,73,30,15.19,scattered clouds,0.0,0
68,68,Cubuk,TR,40.24,33.03,72.00,53,20,14.99,few clouds,0.0,0
85,85,Ribeira Grande,PT,38.52,-28.70,71.60,73,75,14.99,broken clouds,0.0,0
89,89,Morondava,MG,-20.28,44.28,74.75,64,0,11.36,clear sky,0.0,0
106,106,Ponta Do Sol,PT,32.67,-17.10,73.00,70,0,2.46,clear sky,0.0,0
116,116,Tautira,PF,-17.73,-149.15,73.40,83,20,2.24,few clouds,0.0,0


In [4]:
# Add the cities to a marker layer map with a pop-up marker for each city that includes:
# Hotel name
# City
# Country
# Current weather description with the maximum temperature

# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""


# Set parameters to search for a hotel.
from config import g_key
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
 


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [5]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Avarua,CK,71.60,overcast clouds,-21.21,-159.78,Paradise Inn
6,Mandalgovi,MN,70.16,overcast clouds,45.76,106.27,В.Ганбат
15,Pringsewu,ID,73.65,overcast clouds,-5.36,104.97,Urban Style by Front One
44,Liwale,TZ,70.50,light rain,-9.77,37.93,Samoma Lodge
60,Poum,NC,70.12,scattered clouds,-20.23,164.02,Gîte kajeon


In [6]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [9]:
# Add a marker layer map and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Save and upload the new DataFrame as WeatherPy_vacation.csv.

output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")